In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
clauses = []

num_literals = 0
num_clause = 0

with open('../UF250.1065.100/uf250-01.cnf') as file:
    for line in file:
        if(line[0] == 'p'):
            l = line.split()
            num_literals = int(l[2])
            num_clauses = int(l[3])
        if(line[0] != 'p' and line[0] != '%' and line[0] != 'c'):
            clause = [int(digit) for digit in line.split()]
            clauses.append(clause[:-1])

In [3]:
from numpy import linalg as LA

In [4]:
import math

In [5]:
from scipy.stats import norm

In [6]:
def rbf(xi, xj, h):
    return math.exp(-((LA.norm(xi - xj))**2) / h)

In [7]:
def rbf_derivative(xj, xi, h, k):
    return -(2 / h) * k * (xj - xi)

In [8]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [9]:
sigmoid_vector = np.vectorize(sigmoid)

In [10]:
def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

In [11]:
#returns the likelihood of each clause being F for each sample
def likelihood_clauses(num_points, points):
    likelihoods = np.zeros((num_points, num_clauses))
    for i in range(num_points):
        for j in range(num_clauses):
            p = 1
            for literal in clauses[j]:
                if literal > 0:
                    p = p * (1 - sigmoid(points[i][literal-1]))
                else:
                    p = p * sigmoid(points[i][-literal-1])
            likelihoods[i][j] = p
    return likelihoods       

In [28]:
#return the derivative for the i-th sample
def derivative_likelihood(i, likelihoods, num_points, points):
    derivative = np.zeros(num_literals)
    for k in range(num_literals):
        derivative[k] -= 0.5 * norm.pdf(x = points[i][k], loc =5, scale =1) * (points[i][k] - 3) + 0.5 * norm.pdf(x = points[i][k], loc =-5, scale =1) * (points[i][k] + 3) 
        
    for c in range(num_clauses):
        likelihood = likelihoods[i][c]
        for literal in clauses[c]:
            if literal > 0:
                derivative[literal-1] += (likelihood  * sigmoid_derivative(points[i][literal-1]) / ( 1- sigmoid(points[i][literal-1]))) / (1 - likelihood)
            else:
                derivative[-literal-1] += - (likelihood * sigmoid_derivative(points[i][-literal-1]) / sigmoid(points[i][-literal-1])) / (1 - likelihood)
                
    return derivative
    

In [29]:
def satisfied(polarity):
    num_satisfied = 0
    for clause in clauses:
        for literal in clause:
            if literal * polarity[abs(literal) - 1] > 0:
                num_satisfied += 1
                break
    return num_satisfied

In [30]:
def determin_h(points):
    length = int(len(points))
    distance = np.zeros(int(length * (length - 1) / 2))
    m = 0
    for i in range(length):
        for j in range(length):
            if i < j:
                distance[m] = LA.norm(points[i] - points[j])
                m += 1
    return np.median(distance)**2 / math.log(length)

In [31]:
import random

In [32]:
def mixture_gaussian(w, h):
    samples = np.zeros((w, h))
    for i in range(w):
        for j in range(h):
            if random.uniform(0, 1) < 0.5:
                mu = -5
                sigma = 1
                samples[i][j] = np.random.normal(mu, sigma, 1)[0]
            else:
                mu = 5
                sigma = 1
                samples[i][j] = np.random.normal(mu, sigma, 1)[0]
    return samples

In [33]:
def stein_update(num_epochs, num_points):
    points = mixture_gaussian(num_points, num_literals)
    satisfied_clauses = np.zeros(num_epochs)
    for e in range(num_epochs):
        prob = sigmoid_vector(points)
        ave = np.sum(prob, axis=0) / num_points
        polarity = np.zeros(num_literals)
        for i in range(num_literals):
            if ave[i] > 0.5:
                polarity[i] = 1
            else:
                polarity[i] = -1
        print(satisfied(polarity))
        #print(ave[:5])
        satisfied_clauses[e] = satisfied(polarity)
        kernel_matrix = np.zeros((num_points, num_points))
        h = determin_h(points)
        for i in range(num_points):
            for j in range(num_points):
                if i <= j:
                    kernel_matrix[i][j] = rbf(points[i], points[j], h)
                else:
                    kernel_matrix[i][j] = kernel_matrix[j][i]  
        #new_points = np.zeros((num_points, num_literals))
        likelihoods = likelihood_clauses(num_points, points)
        derivatives_likelihood = np.zeros((num_points, num_literals))
        for i in range(num_points):
            derivatives_likelihood[i] = derivative_likelihood(i, likelihoods, num_points, points)
        #print(derivatives_likelihood[0][:5])
        phi_matrix = np.zeros((num_points, num_literals))
        for i in range(num_points):
            phi = 0
            for j in range(num_points):
                phi += kernel_matrix[i][j] * derivatives_likelihood[j] + rbf_derivative(j, i, h, kernel_matrix[i][j])
            phi = phi / num_points
            phi_matrix[i] = phi
        print(LA.norm(derivatives_likelihood))
        print()
        epsilon = 5
        points += epsilon * phi_matrix
    return satisfied_clauses

In [34]:
stein_update(100, 20)

936
67.30741177915557

941
65.83984799870679

945
63.59035146120185

953
60.47610089947325

960
56.65293223788622

968
52.311875702502775

976
48.10140181231267

984
44.19610672081313

983
40.53204978142475

987
37.127098409272946

991
34.03415210964611

992
31.281216597087518

996
28.86428062117431

994
26.74451195539132

1000
24.870083735769388

1004
23.204734132247093

1004
21.727128620046603

1004
20.417753763205116

1006
19.25382621497794

1009
18.21349518573323

1010
17.278843869129965

1012
16.435744711306683

1020
15.671514265878505

1023
14.975792138094764

1023
14.341024251696313

1023
13.762148630896233

1020
13.236090705371176

1020
12.759867511669873

1020
12.328745616204317

1020
11.934653602832332

1020
11.565837993106461

1021
11.210378564960859

1023
10.862406795765585

1023
10.52450256855892

1024
10.202839248484773

1024
9.901662414174869

1023
9.622328977673442

1023
9.364333059549784

1023
9.12631340251093

1024
8.90682737216231

1024
8.704918675691742

1024
8.5200

KeyboardInterrupt: 

In [51]:
x = np.asarray([1, 2])

In [54]:
LA.norm(x)

2.23606797749979

In [72]:
np.random.normal(1, 2, 1)

array([3.03731959])